In [1]:
# import basic libraries
import yt
import pickle
import numpy as np
import matplotlib.pyplot as plt
from astropy import units as u

# import halo catalogue func
from yt.analysis_modules.halo_analysis.api import *

In [3]:
import tempfile
import shutil
import os

# Create temporary directory for storing files
tmpdir = tempfile.mkdtemp()

In [209]:
# load dataset
ds = yt.load('~/../../tigress/cen/LG4_2048_40pc/RD0070/redshift0070')
dd = ds.all_data()

yt : [INFO     ] 2017-07-05 11:07:30,903 Parameters: current_time              = 158.01757996843
yt : [INFO     ] 2017-07-05 11:07:30,905 Parameters: domain_dimensions         = [128 128 128]
yt : [INFO     ] 2017-07-05 11:07:30,908 Parameters: domain_left_edge          = [ 0.  0.  0.]
yt : [INFO     ] 2017-07-05 11:07:30,912 Parameters: domain_right_edge         = [ 1.  1.  1.]
yt : [INFO     ] 2017-07-05 11:07:30,915 Parameters: cosmological_simulation   = 1
yt : [INFO     ] 2017-07-05 11:07:30,917 Parameters: current_redshift          = 4.9999999832278
yt : [INFO     ] 2017-07-05 11:07:30,919 Parameters: omega_lambda              = 0.712
yt : [INFO     ] 2017-07-05 11:07:30,921 Parameters: omega_matter              = 0.288
yt : [INFO     ] 2017-07-05 11:07:30,924 Parameters: hubble_constant           = 0.6933


In [55]:
ds.print_stats()

Parsing Hierarchy : 100%|██████████| 44216/44216 [00:03<00:00, 13670.01it/s]
yt : [INFO     ] 2017-06-29 13:54:55,232 Gathering a field list (this may take a moment.)


level	# grids	       # cells	     # cells^3
----------------------------------------------
  0	   256	       2097152	           128
  1	    60	        547304	            82
  2	   187	       2071552	           128
  3	   898	       9903616	           215
  4	  5193	      55377920	           382
  5	 11014	      58419496	           389
  6	 10277	      39511880	           341
  7	  7393	      20631864	           275
  8	  4828	      10668936	           221
  9	  1558	       5499888	           177
 10	  1220	       5897424	           181
 11	  1332	       7614472	           197
----------------------------------------------
   	 44216	     218241504


t = 1.58017580e+02 = 3.75573046e+16 s = 1.19011917e+09 years

Smallest Cell:
	Width: 2.751e-05 Mpc
	Width: 2.751e+01 pc
	Width: 5.675e+06 AU
	Width: 8.489e+19 cm


In [17]:
ds.field_list

[('all', 'creation_time'),
 ('all', 'dynamical_time'),
 ('all', 'initial_mass'),
 ('all', 'metallicity_fraction'),
 ('all', 'particle_index'),
 ('all', 'particle_mass'),
 ('all', 'particle_position_x'),
 ('all', 'particle_position_y'),
 ('all', 'particle_position_z'),
 ('all', 'particle_type'),
 ('all', 'particle_velocity_x'),
 ('all', 'particle_velocity_y'),
 ('all', 'particle_velocity_z'),
 ('all', 'typeia_fraction'),
 ('enzo', 'Dark_Matter_Density'),
 ('enzo', 'Density'),
 ('enzo', 'Electron_Density'),
 ('enzo', 'H2II_Density'),
 ('enzo', 'H2I_Density'),
 ('enzo', 'HII_Density'),
 ('enzo', 'HI_Density'),
 ('enzo', 'HM_Density'),
 ('enzo', 'HeIII_Density'),
 ('enzo', 'HeII_Density'),
 ('enzo', 'HeI_Density'),
 ('enzo', 'Metal_Density'),
 ('enzo', 'Temperature'),
 ('enzo', 'TotalEnergy'),
 ('enzo', 'x-velocity'),
 ('enzo', 'y-velocity'),
 ('enzo', 'z-velocity'),
 ('io', 'creation_time'),
 ('io', 'dynamical_time'),
 ('io', 'initial_mass'),
 ('io', 'metallicity_fraction'),
 ('io', 'part

In [217]:
# specify boundaries of zoom-in box
# scaling factor multiplied by info from text file 
# units in cm
scaling = 2.22535525e+25 # scales dataset coords to cm

xmin = scaling*0.39319589 * u.cm
ymin = scaling*0.42984636 * u.cm
zmin = scaling*0.41706725 * u.cm

xmax = scaling*0.56298484 * u.cm
ymax = scaling*0.55089246 * u.cm
zmax = scaling*0.56698254 * u.cm

In [5]:
# load halo dataset
halos_ds = yt.load('./LG4_2048_40pc/rd0070/halo_catalogs/catalog/catalog0070_thres160.0.h5')

yt : [INFO     ] 2017-06-29 17:07:14,396 Parameters: current_time              = 3.75573046409e+16
yt : [INFO     ] 2017-06-29 17:07:14,411 Parameters: domain_dimensions         = [2 2 2]
yt : [INFO     ] 2017-06-29 17:07:14,416 Parameters: domain_left_edge          = [ 0.  0.  0.]
yt : [INFO     ] 2017-06-29 17:07:14,423 Parameters: domain_right_edge         = [  2.22535525e+25   2.22535525e+25   2.22535525e+25]
yt : [INFO     ] 2017-06-29 17:07:14,429 Parameters: cosmological_simulation   = 1
yt : [INFO     ] 2017-06-29 17:07:14,435 Parameters: current_redshift          = 4.99999998323
yt : [INFO     ] 2017-06-29 17:07:14,441 Parameters: omega_lambda              = 0.712
yt : [INFO     ] 2017-06-29 17:07:14,446 Parameters: omega_matter              = 0.288
yt : [INFO     ] 2017-06-29 17:07:14,450 Parameters: hubble_constant           = 0.6933


In [6]:
halos_ds.field_list

yt : [INFO     ] 2017-06-29 17:07:17,244 Allocating for 9.178e+03 particles (index particle type 'all')
yt : [INFO     ] 2017-06-29 17:07:17,412 Identified 3.025e+03 octs


[('all', 'particle_identifier'),
 ('all', 'particle_mass'),
 ('all', 'particle_position_x'),
 ('all', 'particle_position_y'),
 ('all', 'particle_position_z'),
 ('all', 'virial_radius'),
 ('halos', 'particle_identifier'),
 ('halos', 'particle_mass'),
 ('halos', 'particle_position_x'),
 ('halos', 'particle_position_y'),
 ('halos', 'particle_position_z'),
 ('halos', 'virial_radius')]

In [49]:
halos_ds.derived_field_list

[('all', 'mesh_id'),
 ('all', 'particle_identifier'),
 ('all', 'particle_mass'),
 ('all', 'particle_ones'),
 ('all', 'particle_position'),
 ('all', 'particle_position_cylindrical_radius'),
 ('all', 'particle_position_cylindrical_theta'),
 ('all', 'particle_position_cylindrical_z'),
 ('all', 'particle_position_relative'),
 ('all', 'particle_position_relative_x'),
 ('all', 'particle_position_relative_y'),
 ('all', 'particle_position_relative_z'),
 ('all', 'particle_position_spherical_phi'),
 ('all', 'particle_position_spherical_radius'),
 ('all', 'particle_position_spherical_theta'),
 ('all', 'particle_position_x'),
 ('all', 'particle_position_y'),
 ('all', 'particle_position_z'),
 ('all', 'particle_radius'),
 ('all', 'particle_spherical_position_phi'),
 ('all', 'particle_spherical_position_radius'),
 ('all', 'particle_spherical_position_theta'),
 ('all', 'virial_radius'),
 ('deposit', 'all_cic'),
 ('deposit', 'all_count'),
 ('deposit', 'all_density'),
 ('deposit', 'all_mass'),
 ('deposi

In [309]:
# Instantiate a catalog using those two paramter files
hc = HaloCatalog(halos_ds=halos_ds, data_ds=ds, output_dir=os.path.join(tmpdir, 'halo_catalog'))
#hc.load()

# create projection plot
p = yt.ProjectionPlot(sp, "x", "Dark_Matter_Density", width=(1.5, 'Mpc'))
p.set_zlim(field="Dark_Matter_Density", zmin=1e23, zmax=1e27)
#p.annotate_halos(hc, factor = "particle_mass")
p.annotate_halos(hc)
p.show()

In [133]:
hc.catalog.sort(key= lambda a : a['virial_radius'].to_ndarray())

In [8]:
satellite = []
nonsatellite = []

In [70]:
for halo1 in hc.halo_list:
    # find first index
    index1 = halo1.quantities.get('particle_identifier')
    
    # create bool to check if this halo isn't satellite
    isSatellite = False
    
    # find parameters of halo1
    x1 = halo1.quantities.get('particle_position_x') * u.cm
    y1 = halo1.quantities.get('particle_position_y') * u.cm
    z1 = halo1.quantities.get('particle_position_z') * u.cm
    radius1 = halo1.quantities.get('virial_radius') * u.cm
    
    # loop through second halo
    for halo2 in hc.halo_list:
        # find second index
        index2 = halo1.quantities.get('particle_identifier')
        
        if index2 <= index1:
            continue
        
        # find parameters of halo2
        x2 = halo2.quantities.get('particle_position_x') * u.cm
        y2 = halo2.quantities.get('particle_position_y') * u.cm
        z2 = halo2.quantities.get('particle_position_z') * u.cm
        radius2 = halo2.quantities.get('virial_radius') * u.cm
        
        # calculate dist between halo centers
        xdist = np.absolute(x2 - x1)
        ydist = np.absolute(y2 - y1)
        zdist = np.absolute(z2 - z1)
        netdist = (xdist**2 + ydist**2 + zdist**2)**0.5
        
        # if netdist < both radii, halo with smaller radius is satellite
        if netdist < radius1 and netdist < radius2:
            # check which radius smaller, if so, add index to list
            if radius2 < radius1:
                satellite.append(index2)
            else:
                satellite.append(index1)
                # set bool to true for later
                isSatellite = True
        
        # if netdist only < rad1, halo2 is satellite
        elif netdist < radius1:
            satellite.append(index2)
        
        # if netdist only < rad2, halo 1 is satellite
        elif netdist < radius2:
            satellite.append(index1)
            # set bool to true for later
            isSatellite = True
        
    # after loop, if satellite bool still false, and halo1 not in satellite
    # list, then add to non-satellite list
    if not isSatellite and index1 not in satellite:
        nonsatellite.append(index1)
    
    

In [421]:
halo1 = hc.halo_list[7675]
halo2 = hc.halo_list[932]

# find first index
index1 = halo1.quantities.get('particle_identifier')

# find parameters of halo1
x1 = halo1.quantities.get('particle_position_x') * u.cm
y1 = halo1.quantities.get('particle_position_y') * u.cm
z1 = halo1.quantities.get('particle_position_z') * u.cm
radius1 = halo1.quantities.get('virial_radius') * u.cm
mass1 = halo1.quantities.get('particle_mass').in_units('Msun')

# find second index
index2 = halo1.quantities.get('particle_identifier')
        
# find parameters of halo2
x2 = halo2.quantities.get('particle_position_x') * u.cm
y2 = halo2.quantities.get('particle_position_y') * u.cm
z2 = halo2.quantities.get('particle_position_z') * u.cm
radius2 = halo2.quantities.get('virial_radius') * u.cm
mass2 = halo2.quantities.get('particle_mass').in_units('Msun')

# calculate dist between halo centers
xdist = np.absolute(x2 - x1)
ydist = np.absolute(y2 - y1)
zdist = np.absolute(z2 - z1)
netdist = (xdist**2 + ydist**2 + zdist**2)**0.5
        


In [235]:
print('distance  = ', netdist)
print('mass1     = ', mass1)
print('mass2     = ', mass2)
print('v_radius1 = ', radius1)
print('v_radius2 = ', radius2)

distance  =  1.585358493209386e+24 cm
mass1     =  9091915.961768644 Msun
mass2     =  19093023.51971414 Msun
v_radius1 =  4.74795826869561e+21 cm
v_radius2 =  4.472863867292526e+21 cm


In [25]:
print(xdist, '\n', ydist, '\n', zdist)

3.1682811356454203e+24 cm 
 8.026487563195402e+23 cm 
 9.789640593412424e+23 cm


In [26]:
print(x1, '\n', x2, '\n', y1, '\n', y2)

1.1963596279206126e+25 cm 
 8.795315143560705e+24 cm 
 1.2021895291189851e+25 cm 
 1.121924653487031e+25 cm


In [382]:
sp1 = ds.sphere([x1.value/scaling, y1.value/scaling, z1.value/scaling], (radius1.value, 'cm'))

In [263]:
x_dist = (sp1["particle_position_x"].in_units('cm')) - (x1.value * np.ones(sp1.size) *u.cm)
y_dist = np.absolute(y2 - y1)
z_dist = np.absolute(z2 - z1)


#conditionalmask = (sp1.)


# small_sp = ds.cut_region(sp1, ["obj['radius'] < 5e20", "obj['radius'] > 1e20"])
# small_sp.mean('Density')

NotImplementedError: cannot evaluate truth value of quantities. Evaluate array with q.value.any(...)

In [177]:
# create list of strings
halolist = []

# fill up lists
for halo1 in hc.halo_list:
    # find first index
    index1 = halo1.quantities.get('particle_identifier')

    # find parameters of halo1
    x1 = halo1.quantities.get('particle_position_x').in_units('cm')
    y1 = halo1.quantities.get('particle_position_y').in_units('cm')
    z1 = halo1.quantities.get('particle_position_z').in_units('cm')
    radius1 = halo1.quantities.get('virial_radius').in_units('cm')
    mass1 = halo1.quantities.get('particle_mass').in_units('Msun')

    # create output string
    output = str(mass1.value)
    output = output + ' ' + str(radius1.value)
    output = output + ' ' + str(x1.value)
    output = output + ' ' + str(y1.value)
    output = output + ' ' + str(z1.value)
    
    # append to list
    halolist.append(output)

In [178]:
halolist

['22426726.039029304 3.9158334864979143e+21 8.621227010597334e+24 1.0964446253739874e+25 8.765728697076966e+24',
 '9091915.961768644 4.74795826869561e+21 1.0310306058491429e+25 1.1657893232940003e+25 1.088322851502297e+25',
 '9394979.827143922 4.7172611224305297e+21 1.1341516415733754e+25 1.1752537751402394e+25 1.1719352224206725e+25',
 '20608342.846675586 4.840551189833346e+21 1.2576035739426941e+25 9.17065987874833e+24 1.0358700479584046e+25',
 '10607235.288730085 3.37071659498959e+21 1.2271502251901438e+25 1.1970132360046873e+25 1.0522308847110742e+25',
 '27881875.616090477 6.464595023483764e+21 1.0557312285784465e+25 1.2645913811220843e+25 1.0047433425487226e+25',
 '10910299.154122373 3.3438992165867786e+21 1.1260911362108288e+25 1.0877124269629785e+25 9.466997580205696e+24',
 '17274640.32736042 6.612922947542092e+21 1.0641010884604393e+25 1.028437008481292e+25 8.694188554557095e+24',
 '11213363.019514661 4.566702594822668e+21 1.2307820765220127e+25 9.853670249153703e+24 1.18945679

In [183]:
# convert halolist to one long string

halostr = halolist[0]
isSkipped = False

for halo in halolist:
    # skip first case
    if isSkipped == False:
        isSkipped = True
        continue
    
    halostr = halostr + '\n' + halo

In [184]:
halostr

'22426726.039029304 3.9158334864979143e+21 8.621227010597334e+24 1.0964446253739874e+25 8.765728697076966e+24\n9091915.961768644 4.74795826869561e+21 1.0310306058491429e+25 1.1657893232940003e+25 1.088322851502297e+25\n9394979.827143922 4.7172611224305297e+21 1.1341516415733754e+25 1.1752537751402394e+25 1.1719352224206725e+25\n20608342.846675586 4.840551189833346e+21 1.2576035739426941e+25 9.17065987874833e+24 1.0358700479584046e+25\n10607235.288730085 3.37071659498959e+21 1.2271502251901438e+25 1.1970132360046873e+25 1.0522308847110742e+25\n27881875.616090477 6.464595023483764e+21 1.0557312285784465e+25 1.2645913811220843e+25 1.0047433425487226e+25\n10910299.154122373 3.3438992165867786e+21 1.1260911362108288e+25 1.0877124269629785e+25 9.466997580205696e+24\n17274640.32736042 6.612922947542092e+21 1.0641010884604393e+25 1.028437008481292e+25 8.694188554557095e+24\n11213363.019514661 4.566702594822668e+21 1.2307820765220127e+25 9.853670249153703e+24 1.1894567927723295e+25\n11213363.01

In [185]:
# output to file 

with open('halo_list_str0070.txt', 'w') as file:
    file.write(halostr)

In [195]:
# check largest x,y,z of all halos (in Mpc) 
# largest halo mass and its virial radius

# declare variables needed
mass_index = 0
largest_mass = 0
v_radius = 0

largest_x = 0
largest_y = 0
largest_z = 0
x_index = 0
y_index = 0
z_index = 0

# find largest parameters
for halo1 in hc.halo_list:
    # find index
    index1 = halo1.quantities.get('particle_identifier')

    # find parameters of halo1
    x1 = halo1.quantities.get('particle_position_x').in_units('Mpc')
    y1 = halo1.quantities.get('particle_position_y').in_units('Mpc')
    z1 = halo1.quantities.get('particle_position_z').in_units('Mpc')
    radius1 = halo1.quantities.get('virial_radius').in_units('cm')
    mass1 = halo1.quantities.get('particle_mass').in_units('Msun')
    
    # compare mass to stored largest mass
    if mass1 > largest_mass:
        # if so, replace mass, index, and v_r values
        largest_mass = mass1
        mass_index = index1
        v_radius = radius1
        
    # compare x val
    if x1 > largest_x:
        largest_x = x1
        x_index = index1
        
    # compare y val
    if y1 > largest_y:
        largest_y = y1
        y_index = index1
        
    # compare z val
    if z1 > largest_z:
        largest_z = z1
        z_index = index1
        
# print results
print('Largest Mass  Index:    ', mass_index)
print('Largest Mass:           ', largest_mass)
print('Largest Mass v_Radius:  ', v_radius)
print('')

print('Largest x Value Index:  ', x_index)
print('Largest x Value:        ', largest_x)
print('')
print('Largest y Value Index:  ', y_index)
print('Largest y Value:        ', largest_y)
print('')
print('Largest z Value Index:  ', z_index)
print('Largest z Value:        ', largest_z)

Largest Mass  Index:     8386.0 dimensionless
Largest Mass:            1319554616983.5603 Msun
Largest Mass v_Radius:   2.7258529196161554e+23 cm

Largest x Value Index:   4981.0 dimensionless
Largest x Value:         6.39637080525251 Mpc

Largest y Value Index:   4981.0 dimensionless
Largest y Value:         7.058640059544112 Mpc

Largest z Value Index:   7331.0 dimensionless
Largest z Value:         7.057597992478424 Mpc


In [192]:
halo1 = hc.halo_list[4981]
x1 = halo1.quantities.get('particle_position_x').in_units('Mpc')
y1 = halo1.quantities.get('particle_position_y').in_units('Mpc')
z1 = halo1.quantities.get('particle_position_z').in_units('Mpc')

print(x1)
print(y1)
print(z1)

6.39637080525251 Mpc
7.058640059544112 Mpc
0.05827257300919312 Mpc


In [359]:
# find radius threshold
# 200 times (the mean density of the universe times
# (1-Omega_Baryon/Omega_matter))
Omega_scale = 1 - 0.16
mean_dens = dd.quantities.weighted_average_quantity('Density', 'ones') 
threshold = 200 * mean_dens * Omega_scale

KeyboardInterrupt: 

In [419]:
# load threshold from file
with open('./LG4_2048_40pc/rd0070/rad_threshold0070', 'rb') as file:
    threshold = pickle.load(file)

In [379]:
# Instantiate a catalog using those two paramter files
hc = HaloCatalog(halos_ds=halos_ds, data_ds=ds, output_dir=os.path.join(tmpdir, 'halo_catalog'))
hc.load()

In [422]:
sp1 = ds.sphere([x1.value/scaling, y1.value/scaling, z1.value/scaling], (radius1.value, 'cm'))
rho_min, rho_max = sp1.quantities.extrema("density")

In [425]:
rp = yt.create_profile(sp1, 'radius', 'density', units = {'radius': 'cm'}, logs = {'radius': True, 'density': True}, n_bins = 256)

In [426]:
print(rp['density'])
print([rp['density'] > threshold])

[  1.87978552e-22   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   6.64895896e-23
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   7.95749952e-23   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   2.44559263e-23
   0.00000000e+00   0.00000000e+00   6.55995979e-23   0.00000000e+00
   0.00000000e+00   1.61298101e-23   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   1.19630295e-23   0.00000000e+00
   8.91010834e-23   2.24464056e-24   8.95993027e-23   0.00000000e+00
   2.08543357e-22   0.00000000e+00   8.59494243e-23   3.25075286e-22
   6.35375198e-23   0.00000000e+00   0.00000000e+00   0.00000000e+00
   1.50870196e-22   3.41127393e-23   1.39248670e-22   0.00000000e+00
   5.11988608e-23   3.71742322e-23

In [420]:
threshold.to('g/cm**3'), rho_min, rho_max

(1.3834219815996882e-22 g/cm**3,
 9.78180472614924e-28 g/cm**3,
 7.182406856085104e-22 g/cm**3)

In [344]:
# attach a sphere object to each halo whose radius extends to twice the radius of the halo
hc.add_callback("sphere", factor=2.0)

In [345]:
# use the sphere to calculate radial profile of density weighted by cell volume in terms of the virial radius
hc.add_callback("profile", "radius", "density", accumulation=True, weight_field="cell_volume", storage="virial_quantities_profiles")

In [340]:
# Define a virial radius for the halo.
hc.add_callback("virial_quantities", ["radius"], profile_storage = "virial_quantities_profiles")

In [341]:
# Save the profiles
hc.add_callback("save_profiles", storage="virial_profiles", output_dir="profiles")

In [346]:
hc.load()

yt : [INFO     ] 2017-07-05 16:12:35,652 Calculating 1D profile for halo 0.


ValueError: bins must be monotonically increasing or decreasing